In [1]:
import json_lines
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from IPython.core import display as ICD

# Introduction

The following notebook manipulates a large 150,000 entry JSONL data file. It does so by first breaking out dictionaries in the name and address fields fields; it converts the dictionary keys into new dataframe columns. It then calculates metrics for the data frame, such as % of occupied fields and unique values within columns. I began by first proving all functions and metrics on a subset dataframe with 100 entries. After successfully completing  a set of tasks on the small dataframe, I'll scale up to the original dataframe. 


In [2]:
# Load data into list, and then load to dataframe
with json_lines.open('../data/ida_wrangling_exercise_data.2017-02-13.jsonl.gz') as f:
    data = [item for item in f]
    df = pd.DataFrame(data)

### Data exploration 

In [148]:
df

,address,dob,email,id,name,phone,record_date,ssn
0,"{'street': '86314 David Pass Apt. 211', 'city'...",1971-06-30,opark@hotmail.com,01d68a4c598a45559c06f4df0b3d82cb,"{'firstname': 'Cynthia', 'lastname': 'Dawson',...",624-869-4610,2006-07-08T09:02:13,xxx-xx-2412
1,"20722 Coleman Villages\nEast Rose, SC 71064-5894",1965-09-09,sperez@armstrong.com,876ff718291d4397bb1e0477ceee6ad9,"{'firstname': 'Tamara', 'lastname': 'Myers'}",1-594-462-7759,2009-03-28T20:22:57,xxx-xx-8025
2,"{'street': '6676 Young Square', 'city': 'New J...",1993-04-12,uortiz@gmail.com,81753097bf7e4e2085982f422bdb9cda,"{'firstname': 'Jamie', 'lastname': 'Alexander'}",472.218.5065x389,2016-08-30T20:31:39,xxx-xx-0568
3,"0932 Gomez Drives\nLeefort, MD 46879-3166",1977-04-14,palmerdiane@yahoo.com,2c2f7154b80f40ca80d08c5adc54ea45,"{'firstname': 'Angela', 'lastname': 'Garcia', ...",1-663-109-4460x1080,2001-02-15T18:50:35,xxx-xx-9825
4,"{'street': '158 Smith Vista', 'city': 'East Sh...",1970-03-19,nancymaxwell@gmail.com,4f5263f339694d068e17ee7fdbb852b8,"{'firstname': 'Jennifer', 'lastname': 'Rodrigu...",233-423-3823,2014-06-21T14:36:01,xxx-xx-9104
5,"{'street': '461 Knapp Unions', 'city': 'Thomps...",1989-12-23,peterkhan@serrano.net,e615757bb70e4c9e8ec532f22a764a3e,"{'firstname': 'Keith', 'lastname': 'Meadows'}",686-305-8557x0623,2012-03-02T10:37:14,xxx-xx-4652
6,"9524 Danielle Burg Apt. 849\nNorth Vickistad, ...",1983-05-09,laurenreilly@lopez-hurst.com,60880b0bcf614ccda8688409ee9d8db3,"{'firstname': 'Shane', 'lastname': 'Garcia'}",721-135-6630,2013-01-19T12:17:21,xxx-xx-5597
7,"407 Patrick Shoal\nWilliamsshire, VI 49526",1989-10-05,wallaceheather@obrien-richardson.com,357cbf915fdb46dfa76c7bd3ea10feff,"{'firstname': 'Charles', 'lastname': 'Bell', '...",528-480-7488x60984,2004-03-03T07:35:59,xxx-xx-7244
8,"{'street': '3697 Mills Estates Apt. 499', 'cit...",1987-06-24,michael18@yahoo.com,bbf4acd3660c4ba1803f396d789e538a,Morgan Morris,09316674472,2000-03-07T03:55:21,xxx-xx-7034
9,NaN,1980-03-20,erinsanford@garcia.com,3422bcb95ad6454f8348087ba25902f0,Bernard Wilkerson,196-454-2484,2016-11-29T21:01:43,xxx-xx-0866


In [4]:
'Number of rows {} '.format(len(df))

'Number of rows 150000 '

This initial dataframe has 150,000 entries. Some of the fields (e.g. address and name) are recorded as dictionaries and strings. Let's start working with a subset (first 100 entries ) of the data frame. Truncating the dataframe will speed up initial code experimentation. 

### Operating on subset of data 

In [135]:
# Subset wtih first 100 entries of dataframe
df_sub = df[0:100].copy()

## 1. Make a list of all of the nested named fields that appear in any record. Concatenate nested field names using a period '.' to defind named fields for nested records. Present the list in alphabetical order.

In [136]:
def expand_dict(dframe, dict_names):
    """This function will take a dataframe and dictionary list as inputs. 
    It will convert dictionary keys into seperate columns within the same dataframe by invoking the apply method.
    It returns a dataframe with concatenated columns"""

    # Creates an empty dataframe
    df_nested = pd.DataFrame()

    # Iterate through list of dictionaries
    for item in dict_names:
        # Expand dictionary keys into columns
        df_nested = pd.concat([df_nested, dframe[item].apply(pd.Series).add_prefix(
            (item + '.'))], axis=1)
        # Drops extra generated column from apply method
        df_nested.drop(item + '.0', axis=1, inplace=True)
        # Add original nested column
        df_nested[item] = dframe[item]
        # Replace nested fields with NaN, otherwise leave string value
        df_nested[item] = df_nested[item].apply(lambda x: np.nan if type(x) == dict else x)

    return df_nested

In [137]:
#Invoke the function above 
df_sub = expand_dict(df_sub, ['address', 'name'])
df_sub.head()

,address.city,address.state,address.street,address.zip,address,name.firstname,name.lastname,name.middlename,name
0,Hoodburgh,RI,86314 David Pass Apt. 211,83973,NaN,Cynthia,Dawson,Claire,NaN
1,NaN,NaN,NaN,NaN,"20722 Coleman Villages\nEast Rose, SC 71064-5894",Tamara,Myers,NaN,NaN
2,New Julie,UT,6676 Young Square,73125,NaN,Jamie,Alexander,NaN,NaN
3,NaN,NaN,NaN,NaN,"0932 Gomez Drives\nLeefort, MD 46879-3166",Angela,Garcia,Alexis,NaN
4,East Sharonstad,ME,158 Smith Vista,42483,NaN,Jennifer,Rodriguez,NaN,NaN


In [138]:
# An alphabetical order list of the fields 
sorted(list(df_sub.columns))

['address',
 'address.city',
 'address.state',
 'address.street',
 'address.zip',
 'name',
 'name.firstname',
 'name.lastname',
 'name.middlename']

## 2. Answer the following questions for each field in your list from question 1.

- What percentage of the records contain the field?
- What are the five most common values of the field?

#### What percentage of the records contain the field?

In [139]:

def col_percent(dframe):
    '''A funciton to loop through columns and return the percentage of populated items.'''
    for item in dframe:
        print('{} = {:.1f}%'.format(item, 100 *
                                    dframe[item].count() / len(dframe)))

In [140]:
# Invoke the function above 
col_percent(df_sub)

address.city = 41.0%
address.state = 41.0%
address.street = 41.0%
address.zip = 41.0%
address = 52.0%
name.firstname = 66.0%
name.lastname = 66.0%
name.middlename = 26.0%
name = 34.0%


#### What are the five most common values of the field?

In [141]:
# Five most common values in a column
def common_values(df, num):
    '''Function takes a dataframe and int as input, returns int number of common values'''
    for item in df:
        # Prints common values as a dataframe
        ICD.display(pd.DataFrame(df[item].value_counts().head(num)) )

In [142]:
common_values(df_sub,5)

,address.city
Nolanmouth,1
Josephshire,1
Williamtown,1
Perezhaven,1
Christopherborough,1


,address.state
UT,4
RI,4
WY,3
HI,2
VI,2


,address.street
3852 Garcia Rapids,1
01737 Hailey Drives Suite 056,1
56833 Baker Branch,1
93406 Justin Spur,1
461 Knapp Unions,1


,address.zip
44661,1
15884,1
78860,1
19394-0990,1
77432-0490,1


,address
"090 Laura Islands\nLake Jennifer, OR 78668-8235",1
"31979 Wendy Hollow Suite 911\nNew Jeremy, VI 00557",1
"82863 Paul Manor Suite 652\nLake James, AR 69358",1
"577 Brown Skyway Suite 354\nShaneview, VI 02325",1
"65447 Perry Fall\nStephanieton, GU 78143",1


,name.firstname
Robert,3
Steven,2
Shane,2
Eric,2
Tiffany,2


,name.lastname
Davis,3
Garcia,2
Chapman,2
Hall,2
Rodriguez,2


,name.middlename
David,2
Jerry,1
Justin,1
Jennifer,1
Paul,1


,name
Bernard Wilkerson,1
Christine Mcguire,1
David Thomas,1
Lauren Turner,1
Terry Nguyen,1


## 3. How many distinct first names appear in this data set? Explain your procedure for identifying distinct first names.

Used the unique() method on the 'name.namefirst' column. With more time I would have parsed out the first names strings in the 'name' column. 

In [143]:
df_sub['name.firstname'].unique()

array(['Cynthia', 'Tamara', 'Jamie', 'Angela', 'Jennifer', 'Keith',
       'Shane', 'Charles', nan, 'Amanda', 'Adrian', 'Tiffany', 'Peter',
       'Steven', 'Lindsay', 'Sara', 'Amy', 'Christopher', 'Melissa',
       'Crystal', 'Mallory', 'Elizabeth', 'Richard', 'Jonathan', 'Nicole',
       'Michael', 'Robert', 'Ryan', 'Mary', 'Brooke', 'Daniel', 'Calvin',
       'Brandon', 'Laurie', 'Ashley', 'Brent', 'Kevin', 'Eric', 'Deborah',
       'Christine', 'Joseph', 'Donald', 'Jeff', 'Nathan', 'Martin', 'Lisa',
       'Ricky', 'Casey', 'Stacy', 'Matthew', 'Cassidy', 'Jenna', 'Mark',
       'Brad', 'Stephanie', 'John', 'Lauren', 'Veronica'], dtype=object)

In [144]:
len(df_sub['name.firstname'].unique())

58

## 4. How many distinct street names appear in this data set? Explain your procedure for identifying distinct street names.

## 5. What are the 5 most common US area codes in the phone number field? Explain your approach to identify the US area codes in this data set.
To do: remove special charectors. Remove extensions from end starting with x, then remove last 7 numbers, then last three will be area code. 